In [88]:
import hmac
import hashlib

# Secret key and message
key = b"supersecretkey"
message = b"Hello, world!"

# Create HMAC using SHA-256
hmac_object = hmac.new(key, message, hashlib.sha1)
hash = hashlib.sha1(message)
# hmac_digest = hmac_object.hexdigest()

print("Keyed Hash (HMAC-SHA1):", hmac_object.hexdigest())
print("Un-Keyed Hash (HMAC-SHA1):", hash.hexdigest())

Keyed Hash (HMAC-SHA1): 4cb62fcbc3810327973a632c2fb1c8c20d57ea82
Un-Keyed Hash (HMAC-SHA1): 943a702d06f34599aee1f8da8ef9f7296031d699


In [2]:
import time, base64, os, re,json5, pprint,random , math, hashlib
import pandas as pd
from selenium import webdriver                                # Controls the browser
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
import pandas as pd
import numpy as np
from io import StringIO

In [ ]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\data\13082025"

# datastr = Helper.read_file(path)
# df = pd.read_html(StringIO(datastr))[0]
def fix_mojibake(text:str)->str:
    try:
        return text.encode("latin1").decode("utf-8")
    except:
        return text  # If it fails, return original

In [ ]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\data\13082025"
banks = ['Bank of Baroda', 'Bank of India', 'Bank of Maharashtra', 'Canara Bank', 'Central Bank of India', 'Indian Bank', 'Indian Overseas Bank', 'Punjab and Sind Bank', 'Punjab National Bank', 'State Bank of India', 'Uco Bank', 'Union Bank of India']

for bank in banks:
    
    print(bank)
    dirs  =os.path.join(path,bank, "save_html")
    if os.path.exists(dirs):
        contents = os.listdir(dirs)
        for content in contents:
            html_path = os.path.join(dirs,content)
            if not os.path.isfile(html_path):
                continue
            df  = pd.read_html(html_path)[0]
            df = df.map(lambda x: x.lower() if isinstance(x,str) else x)
            df = df.map(lambda x:str(x))
            print(content)
            if df.empty:
                continue
            id = " ".join(df.iloc[0,:].to_list())
            byte_id = id.encode(encoding="utf-8")
            print(hashlib.sha1(byte_id).hexdigest())

    # for content in contents:
    #     html_path = os.path.join(root,content)
        
    #     df  = pd.read_html(html_path)[0]
    #     df = df.map(lambda x: x.lower() if isinstance(x,str) else x)
    #     df = df.map(lambda x:str(x))
    #     print(content)
    #     id = " ".join(df.iloc[0,:].to_list())
    #     byte_id = id.encode(encoding="utf-8")
    #     print(hashlib.sha1(byte_id).hexdigest())

In [23]:
path = r"C:\Users\kaustubh.keny\Projects\JSON25\scrape_output\data\13082025\Punjab and Sind Bank\save_html\content_2.html"
dataStr = Helper.read_file(path)
dataStr = re.sub(r'<tr[^>]*>\s*(?:&nbsp;|\s)*</tr>|Â ',"",dataStr, re.IGNORECASE)
raw_html = re.sub(r'(Â[\xa0\s]*)+', ' ', dataStr)

df = pd.read_html(dataStr)[0]

C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_2972\3130505306.py:3: DeprecationWarning: 'count' is passed as positional argument
  dataStr = re.sub(r'<tr[^>]*>\s*(?:&nbsp;|\s)*</tr>|Â ',"",dataStr, re.IGNORECASE)
C:\Users\kaustubh.keny\AppData\Local\Temp\ipykernel_2972\3130505306.py:6: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(dataStr)[0]


In [24]:
df.to_excel("check.xlsx")

In [ ]:
import json5
from dateutil.parser import parse
import re

path = r'example.json5'

def extract_date(text: str) -> str:
    if not isinstance(text, str):
        print("Invalid data type. Expected string.")
        return ""
    
    output_format = "%Y%m%d"
    date_pattern = r"(\d{2}[.\-/]+\d{2}[.\-/]+\d{4}|\d{1,2}\s*(?:th|st|rd|nd)\s*[A-Za-z]+\s*\d{4}|\d{2}[\.\-\/]+[A-Za-z]+[\.\-\/]+\d{4}|\d{2}\s*[A-Za-z]+\s*\d{4})"
    matches = re.findall(date_pattern, text, re.IGNORECASE)
    
    if matches:
        date_str = " ".join(matches)
        dt_object = parse(date_str, fuzzy=True)
        return dt_object.strftime(output_format)
    
    # print("Date not found.")
    return text

with open(path, "r+", encoding="utf-8") as file:
    data = json5.load(file)

records = data.get("records", [])

for record in records:
    
    print(record["bank_name"])
    response_data = record.get("response", {})
    if not response_data:
        continue

    extracted_dates = {}

    for block in response_data.values():
        scrape_data = block.get("scrape_data", {})
        if not scrape_data:
            continue

        cleaned_data = {key: extract_date(value) for key, value in scrape_data.items()}
        extracted_dates.update(cleaned_data)

    print(extracted_dates)
    record["response"] = extracted_dates

with open("up20250801.json5", "r+", encoding="utf-8") as file:
   json5.dump(data,file)

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random
import time
import math


def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))


def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()


def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()


# ==== SETUP CHROME WITH GOOGLE PROFILE ====
profile_path = r"C:\Users\rando\AppData\Local\Google\Chrome\User Data"
profile_dir = "Profile 1" # or "Default"

options = uc.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

# ==== GO TO SITE ====
driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

# ==== SIMULATE HUMAN BEHAVIOR ====
human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()

import undetected_chromedriver as uc 

profile_path = r"C:\Users\kaustubh.keny\AppData\Local\Google\Chrome\User Data"
profile_dir = "Default" # Change to match your actual one

options = uc.ChromeOptions()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
# options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(driver_executable_path=r"C:\Users\kaustubh.keny\chromedriver-win64\chromedriver.exe",options=options)
driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

# import scrapy

# class UnionBankSpider(scrapy.Spider):
#     name = "unionbank_tables"
#     allowed_domains = ["unionbankofindia.co.in"]
#     start_urls = ["https://www.unionbankofindia.co.in/en/details/rate-of-interest"]

#     def parse(self, response):
#         tables = response.css("table").getall()
#         self.logger.info(f"Found {len(tables)} tables")
#         for idx, table in enumerate(tables):
#             yield {
#                 "table_index": idx,
#                 "html": table
#             }

